In [161]:
import pybaseball as pyball
from pybaseball import schedule_and_record
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pybaseball import statcast
from pybaseball import batting_stats
from pybaseball import team_batting
from pybaseball import cache
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score




In [162]:
batters_before = batting_stats(2022, qual=200)

In [163]:
batters_before

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA,L-WAR
0,15640,2022,Aaron Judge,NYY,30,157,570,696,177,87,...,118.4,246,0.609,404,0.169,0.287,NaN,NaN,NaN,11.4
7,11493,2022,Manny Machado,SDP,29,150,578,644,172,102,...,112.4,219,0.490,447,0.126,0.243,NaN,NaN,NaN,6.7
8,9777,2022,Nolan Arenado,STL,31,148,557,620,163,90,...,111.4,190,0.389,489,0.155,0.241,NaN,NaN,NaN,7.2
6,5361,2022,Freddie Freeman,LAD,32,159,612,708,199,129,...,112.3,248,0.480,517,0.122,0.206,0.313,0.538,0.403,6.6
2,9218,2022,Paul Goldschmidt,STL,34,151,561,651,178,102,...,112.3,200,0.469,426,0.196,0.295,0.261,0.482,0.367,7.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,11200,2022,Kole Calhoun,TEX,34,125,388,424,76,49,...,109.9,123,0.482,255,0.116,0.301,NaN,NaN,NaN,-1.1
346,10071,2022,Jonathan Villar,- - -,31,59,202,220,42,31,...,109.5,58,0.397,146,0.114,0.268,NaN,NaN,NaN,-0.7
270,11680,2022,Elias Diaz,COL,31,105,351,381,80,51,...,111.8,107,0.393,272,0.134,0.244,NaN,NaN,NaN,-1.5
297,1744,2022,Miguel Cabrera,DET,39,112,397,433,101,86,...,111.0,115,0.382,301,0.149,0.270,NaN,NaN,NaN,-1.5


In [164]:
batters = batting_stats(2023, qual = 200)

In [165]:
batters[batters['IDfg'] == 12532]

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA,L-WAR
327,12532,2023,Kolten Wong,SEA,32,67,194,216,32,24,...,107.9,38,0.253,150,0.158,0.247,NaN,NaN,NaN,-1.0


In [166]:
bat2022to2023 = pd.merge(left = batters[['IDfg', 'Name', 'WAR']], right = batters_before[['IDfg', 'WAR']], on = "IDfg", how = "inner").rename(columns = {"WAR_x" : "WAR_2023", "WAR_y" : "WAR_2022"})

In [167]:
#filter so that we find all seasons where a batter's war changes greater than 0.1 compared to a previous season. Arbitrary cutoff at 0.7 to ensure that we don't many people's seasons fluctuating by small amounts
outliers = bat2022to2023[(abs(bat2022to2023['WAR_2023'] - bat2022to2023['WAR_2022']) > abs(bat2022to2023['WAR_2022']) * 0.1) & (bat2022to2023['WAR_2022'] >= 0.7)]

In [168]:
#identify who improved and turn into a serial
improvers = outliers[outliers['WAR_2023'] > outliers['WAR_2022']]['IDfg']
regressors = outliers[outliers['WAR_2023'] < outliers['WAR_2022']]['IDfg']

In [169]:
#label in original dataset who improved/regressed
batters_before.loc[batters_before['IDfg'].isin(improvers), 'regressor'] = 1
batters_before.loc[batters_before['IDfg'].isin(regressors), 'regressor'] = 2

In [170]:
#split labeled data into features and labels
x = batters_before.drop("regressor", axis=1).select_dtypes(include=['number']).iloc[:,3:].dropna(axis=1)

y = batters_before['regressor'].fillna(0)

In [171]:


x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


In [172]:
import xgboost as xgb

# Create an XGBoost model instance
model = xgb.XGBClassifier(objective='multi:softmax', num_class=3)  # For classification tasks; use xgb.XGBRegressor() for regression

# Fit the model to the training data
model.fit(x_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_class=3,
              num_parallel_tree=None, objective='multi:softmax', ...)

In [173]:

y_pred = model.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)

print("Accuracy:", accuracy)

Accuracy: 0.7083333333333334
